# Turn series of images into video 
Jupyter Notebook Script to turn series of photos into a one photo per day video compilation
- Resizes photos to 1080p
- Add text to photos (Day, Location, Country)
- Add Icons to photos (Flag of visited country, Map Marker)
- Saves each photo in directory before compiling them in a video

**Ideas:**
- Find nice design for text and flag
- calendar symbol instead of "Day"

**TODOs:**
- Get maximum one photo per day (maybe skip days with not so nice pictures)
    - Find best frame within each photo
- Make sure that fotos are ordered by date!

**What text to show on each photo**
- Day Counter
- Date?
- Location, Country (Flag, and/or Map?)

In [1]:
import cv2 # image and video processing
import os # handling of filenames and directories
import exifread # extract meta data from photos (date photo taken)
import datetime # handling dates and days
import re # for extracting date from filename
import pandas as pd # for processing excel spread sheet with location info
from PIL import Image, ImageDraw, ImageFont # for nicer fonts when adding text

In [3]:
# directory where pictures are stored
input_dir = 'photos_dev' # folder with photos
flag_dir = 'country_flags' # folder with png (transparent) photos of flags for each country visited 
output_dir = 'photos_processed' # output folder of rescaled photos
outputtxt_dir = 'photos_text' # output folder of final photos with text

# Check if output directories exit
# if so, delete all jpg photos within directory
# if not, create directory
if os.path.exists(output_dir):
    for file_name in os.listdir(output_dir):
        if file_name.endswith('.jpg'):
            os.remove(os.path.join(output_dir, file_name))
else:
    os.makedirs(output_dir)
    
if os.path.exists(outputtxt_dir):
    for file_name in os.listdir(outputtxt_dir):
        if file_name.endswith('.jpg'):
            os.remove(os.path.join(outputtxt_dir, file_name))
else:
    os.makedirs(outputtxt_dir)  

In [3]:
# Read excel spreadsheet
# Format: Date,Day,location1,location2,height
df = pd.read_excel('travel_data.xlsx')
df = df.set_index('Day') # set Day as primary key
df['Date'] = df['Date'].dt.date # convert dates to datetime
start_date = datetime.date(2022,4,4) # Day counter

In [4]:
# Video settings
# video_sz = (1280, 720) # 720p
video_sz = (1920, 1080) # 1080p
video_name = 'output_video_v3.mp4'
fps = 5.0 # frames per second
photo_time = 1.5 # in seconds
photo_frames = int(fps*photo_time)

In [5]:
def prepare_image(img, sz, ykeep = 0.5):
    """
    Resize and crop or pad image to desired size

    Args:
        img: np.ndarray
            input Image
        sz: int tuple
            desired size
        ykeep: float
            what part of image should be kept if being cropped
            
    Returns:
        img: np.ndarray
            resized Image
    """
        
    # shape of image
    height, width, _ = img.shape

    # Resizing the image to width by keeping aspect ratio
    scale = width/sz[0]
    height_new = int(height / scale)
    dim_new = (sz[0],height_new)                
    img = cv2.resize(img, dim_new)

    # Check
    # height, width, _ = img.shape
    # print(f'resized: h={height} w={width}')

    # Crop image on height (take the center)
    if height_new>sz[1]:
        
        d = height_new - sz[1]
        idx1 = int(d*ykeep)
        idx2 = height_new-int((1-ykeep)*d)
        diff = (idx2-idx1)-sz[1] # compensate for uneven pixel height
        idx2 = idx2-diff
        img = img[idx1:idx2, :] 
        
        #height, width, _ = img.shape
        #print(f'-> cropping: h={height} w={width}')

    # Pad with zeros if image is height is too small
    elif height_new<sz[1]: 
        diff = int((sz[1]-height_new)/2)
        diff_corr = 2*diff+height_new-sz[1] # compensate for uneven pixel height
        img = cv2.copyMakeBorder(img,diff,diff-diff_corr,0,0,cv2.BORDER_CONSTANT,value=[0,0,0])
        #height, width, _ = img.shape
        #print(f'-> zero-padding: h={height} w={width}')

    # asseert that exported image size is of desired resolution
    if img.shape[0] != sz[1]: 
        raise AssertionError(f'Height of image {img.shape[0]} does not match desired height {sz[1]}')
    if img.shape[1] != sz[0]:
        raise AssertionError(f'Width of image {img.shape[1]} does not match desired width {sz[0]}')

    return img

In [6]:
def get_photo_date(filename):
    """
    Return date of image taken

    TODO: if date cannot be extracted from photo or filename, catch error

    Args:
        filename: str
            filename/to/image
    Returns:
        date: datetime.date
            date of image taken
    """
    # get meta information about image
    with open(filename, 'rb') as f:
        tags = exifread.process_file(f)
        if 'EXIF DateTimeOriginal' in tags:
            # extract the time the photo was taken
            time_taken_str = tags['EXIF DateTimeOriginal'].printable
            time_taken = datetime.datetime.strptime(time_taken_str, '%Y:%m:%d %H:%M:%S')
            #print(f'{filename} was taken on {time_taken}')
            #print(time_taken.strftime('%d/%m/%y'))
            return time_taken.date()
        else:
            match = re.search(r'\d{8}', filename)
            date = datetime.datetime.strptime(match.group(), '%Y%m%d').date()
            print(f'>> {filename} does not have a DateTimeOriginal tag. Automatically extracted date {date}')

            return date
        
def extract_info(date,df):
    """
    Extract meta information from excel spreadsheet of photo given date

    Args:
        date: datetime.date
            Date of photo taken
        df: pandas.core.frame.DataFrame
            DataFrame table of travel locations
    Returns:
        country: str
            name of country
        loc: str
            name of more precise location (e.g., closest city)
        ykeep: float
            what part of the image should be kept (if necessary)
            0: keep upper part
            1: keep bottom part
            0<x<1: keep respective relative area (i.e., 0.5 keeps center of photo)
    """
    idx_date = df['Date']==date
    if sum(idx_date) != 1:
        raise Exception('Date found less or more than once in excel spread sheet!')
    country = df.loc[idx_date,'country'].values[0]
    loc = df.loc[idx_date,'location1'].values[0]
    ykeep = df.loc[idx_date,'height'].values[0]
    
    if ykeep<0 or ykeep>1:
        raise Exception('ykeep is not bounded by [0 1]!')
    
    return country,loc,ykeep

In [7]:
def add_text(img,text,xpos=10,ypos=10,fontScale=2,thickness=1):
    """ [DEPRECATED] USES openCV function (cannot set Font)
    Add text to image

    Args:
        img: np.ndarray
            input Image
        text: str
            text to be added
        xpos: int
            x position of text
        ypos: int
            y position of text
        fontScale: int
            scale of text
        thickness: int
            thickness of text
    Returns:
        img: np.ndarray
            Image with text
    """
    font                   = cv2.FONT_HERSHEY_SIMPLEX
    bottomLeftCornerOfText = (xpos,ypos)
    fontColor              = (255,255,255)
    lineType               = cv2.LINE_AA
    
    cv2.putText(img,text,bottomLeftCornerOfText, 
       font, fontScale, fontColor, thickness, lineType)
    
    return img

def add_text_PIL(file_name,input_dir,output_dir,text_large,text_small):
    """ 
    Add text to image using Pillow Library (better due to more Fonts)

    Args:
        file_name: str
            name of image
        input_dir: str
            directory  where image is located
        output_dir: str
            directory where image with text will be saved
        text_large: str
            large text to be displayed
            here: "Day XX"
        text_small: str
            smaller text to be displayed (below text_large)
            here: "Location, Country"
    Returns:
        None
    """
    filename = os.path.join(input_dir, file_name)
    img = Image.open(filename)
    draw = ImageDraw.Draw(img)

    font_name = 'fonts/ShantellSans-Bold.ttf'
    font_size = 100
    font = ImageFont.truetype(font_name, font_size)
    #draw.text((10,880),text_large,font=font,fill='rgb(255,255,255)',stroke_width=1,stroke_fill='rgb(0,0,0)')
    draw.text((10,880),text_large,font=font,fill='rgb(255,255,255)')
    font_name = 'fonts/ShantellSans-SemiBold.ttf'
    font_size = 50
    font = ImageFont.truetype(font_name, font_size)
    draw.text((60,1000),text_small,font=font,fill='rgb(255,255,255)')

    # save the image to the output directory
    output_file = os.path.join(output_dir, file_name)
    img.save(output_file)
    print('AddedText: Saved ' + file_name)

In [8]:
def get_image_scaled(filename,new_heigth):
    """
    Get an image given desired height (keeps aspect ratio)
    used for png images with transparent background

    Args:
        filename: str
            location/of/image
        new_heigth: int
            desired heigth in pixels of image
    Returns:
        img: np.ndarray
            rescaled image 
    """
    
    # to load in png with additional alpha channel
    img = cv2.imread(filename,cv2.IMREAD_UNCHANGED) 
    
    if img is None:
        raise Exception('Image ' + filename + ' not found')
    
    # shape of image
    height, width, _ = img.shape
    
    #print(f'old: {height} {width}')

    # Resizing the image to width by keeping aspect ratio
    scale = new_heigth/height
    dim_new = (int(width*scale),new_heigth)                
    img = cv2.resize(img, dim_new)
    
    #print(f'new: {img.shape[0]} {img.shape[1]}')
    
    return img

def add_img(img_1,img_2,x_offset=50,y_offset=50):
    """
    Add transparent image (png) to larger photo image

    Args:
        img_1: np.ndarray
            image to be added, result of get_image_scaled
            Note: alpha channel required in addition to its RGB channels
        img_2: np.ndarray
            photo to be overlaid by alpha img
        x_offset: int
            x location of img_alpha within img
        y_offset: int
            y location of img_alpha within img
    Returns:
        img: np.ndarray
            photo with added img_alpha
    """

    # add alpha image to photo
    y1, y2 = y_offset, y_offset + img_1.shape[0]
    x1, x2 = x_offset, x_offset + img_1.shape[1]

    # take care of transparent background
    # combine both picture weighted by alpha values
    alpha_1 = img_1[:, :, 3] / 255.0
    alpha_2 = 1.0 - alpha_1

    # Go through all channels
    for c in range(0, 3):
        img[y1:y2, x1:x2, c] = (alpha_1 * img_1[:, :, c] +
                                  alpha_2 * img_2[y1:y2, x1:x2, c])
    
    return img

In [9]:
# Process Images
# - go through each image in input directory
# - resize and crop or pad image to video size
# - extrat date of photo taken
# - add text
# - save transformed photo

# get MapMarker
mapmarker_img = get_image_scaled('resources/Map_marker.png',int(video_sz[1]*0.035))
prev_date = start_date

for file_name in os.listdir(input_dir):
    if file_name.endswith('.jpg'):

        # read the image
        filename = os.path.join(input_dir, file_name)
        
        # get date when image was taken
        curr_date = get_photo_date(filename)
        
        # check whether date is after date of previous photo
        if prev_date>curr_date:
            raise Exception('Photo ' + filename + ' date not sorted!')
        
        curr_day = (curr_date-start_date).days
        prev_date = curr_date
        
        # get meta information about day from dataframe
        country,loc,ykeep = extract_info(curr_date,df)
                
        # prepare image
        img = cv2.imread(filename)
        img = prepare_image(img, video_sz, ykeep)
        
        # get country flag
        flag_img = get_image_scaled(os.path.join(flag_dir, country + '.png'),int(video_sz[1]*0.05))
        
        # Add country flag and MapMarker
        img = add_img(flag_img,img,20,840)
        img = add_img(mapmarker_img,img,20,1015)
        
        # save the image to the output directory
        output_file = os.path.join(output_dir, file_name)
        cv2.imwrite(output_file, img)
        print('Processed: Saved ' + file_name)
        
        # Add Text
        text_large = 'Day ' + str(curr_day)
        if loc==country:
            text_small = loc # Singapore show only city
        else:
            text_small = loc + ', ' + country
        add_text_PIL(file_name,output_dir,outputtxt_dir,text_large,text_small)

Processed: Saved PXL_20220404_122730396.jpg
AddedText: Saved PXL_20220404_122730396.jpg
Processed: Saved PXL_20220405_133836774.jpg
AddedText: Saved PXL_20220405_133836774.jpg
Processed: Saved PXL_20220406_061928764.MP.jpg
AddedText: Saved PXL_20220406_061928764.MP.jpg
Processed: Saved PXL_20220407_113947773.jpg
AddedText: Saved PXL_20220407_113947773.jpg
Processed: Saved PXL_20220408_034225512.jpg
AddedText: Saved PXL_20220408_034225512.jpg
Processed: Saved PXL_20220409_093839860.jpg
AddedText: Saved PXL_20220409_093839860.jpg
Processed: Saved PXL_20220410_094057789.jpg
AddedText: Saved PXL_20220410_094057789.jpg
Processed: Saved PXL_20220411_082942412.jpg
AddedText: Saved PXL_20220411_082942412.jpg
Processed: Saved PXL_20220412_045052526.MP.jpg
AddedText: Saved PXL_20220412_045052526.MP.jpg
Processed: Saved PXL_20220413_051713537.jpg
AddedText: Saved PXL_20220413_051713537.jpg
Processed: Saved PXL_20220414_060135292.jpg
AddedText: Saved PXL_20220414_060135292.jpg
Processed: Saved PXL

In [10]:
# Make Video
# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video_writer = cv2.VideoWriter(video_name, fourcc, fps, video_sz)

# go through each image in input directory
print('Making video ...')
for file_name in os.listdir(outputtxt_dir):
    if file_name.endswith('.jpg'):
        img = cv2.imread(os.path.join(outputtxt_dir, file_name))
        print(f'Processing {file_name} ...')
        for i in range(photo_frames):
            video_writer.write(img)
            
cv2.destroyAllWindows()
video_writer.release()
print('Video done.')

Making video ...
Processing PXL_20220404_122730396.jpg ...
Processing PXL_20220405_133836774.jpg ...
Processing PXL_20220406_061928764.MP.jpg ...
Processing PXL_20220407_113947773.jpg ...
Processing PXL_20220408_034225512.jpg ...
Processing PXL_20220409_093839860.jpg ...
Processing PXL_20220410_094057789.jpg ...
Processing PXL_20220411_082942412.jpg ...
Processing PXL_20220412_045052526.MP.jpg ...
Processing PXL_20220413_044028133.jpg ...
Processing PXL_20220413_051713537.jpg ...
Processing PXL_20220414_060135292.jpg ...
Processing PXL_20220415_040601404.jpg ...
Processing PXL_20220416_041257069.jpg ...
Processing PXL_20220417_060853938.jpg ...
Processing PXL_20220418_055314760.jpg ...
Processing PXL_20220419_044613581.jpg ...
Processing PXL_20220420_064531355.jpg ...
Processing PXL_20220421_061607742.jpg ...
Processing PXL_20220422_034951694.jpg ...
Processing PXL_20220423_054037289.PORTRAIT.jpg ...
Processing PXL_20220424_121652581.jpg ...
Processing PXL_20220425_114744022.jpg ...
Pr